# Analysis Code

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import statsmodels.api as sm

<p>Now based off the computed differences of the different criteria. We now will use a logistic regression to analyze
where if these differences are able to predict the probability of choosing the left monster (could also be right, this choice
is just arbitrary). In simpler terms, by using this statistical model, can we see if these criteria actually play a role in how they choose a monster.</p> 

<p>To start off, let's just run a logistic regression on one participant</p>

In [2]:
data = pd.read_csv('Main_df.csv')
data.drop(data.columns[0], axis=1) #there was a duplicate of the trial_index column called unnamed, dropping it here

,rt,response,Chosen_Monster,trial_type,trial_index,time_elapsed,design_file,leftstim,rightstim,Color_d,Cuteness_d,Emotions_d,Personal_Preference_d,Shape_d,Size_d
0,4267,ArrowRight,0,html-keyboard-response,4,81105,PP050.csv,obj14.png,obj03.png,54.0,335.0,676.0,543.0,283.0,205.0
1,3232,ArrowRight,0,html-keyboard-response,5,85769,PP050.csv,obj07.png,obj06.png,325.0,-66.0,126.0,228.0,334.0,-468.0
2,3657,ArrowLeft,1,html-keyboard-response,6,90834,PP050.csv,obj09.png,obj11.png,155.0,-207.0,-97.0,-373.0,115.0,-614.0
3,2870,ArrowLeft,1,html-keyboard-response,7,95117,PP050.csv,obj03.png,obj13.png,-109.0,-381.0,-611.0,-482.0,-343.0,102.0
4,5316,ArrowLeft,1,html-keyboard-response,8,101852,PP050.csv,obj02.png,obj07.png,-56.0,-74.0,200.0,-168.0,-72.0,214.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,1898,ArrowLeft,1,html-keyboard-response,90,377049,PP050.csv,obj05.png,obj01.png,-61.0,77.0,320.0,61.0,71.0,532.0
87,600,ArrowLeft,1,html-keyboard-response,91,379066,PP050.csv,obj01.png,obj00.png,-220.0,-676.0,-446.0,-397.0,-295.0,-294.0
88,1150,ArrowLeft,1,html-keyboard-response,92,381645,PP050.csv,obj01.png,obj09.png,-654.0,-344.0,-573.0,-268.0,64.0,-66.0
89,1441,ArrowRight,0,html-keyboard-response,93,384498,PP050.csv,obj11.png,obj06.png,224.0,273.0,609.0,533.0,-50.0,284.0


In [ ]:
#run the logistic regression model

#defining variables
independent_vars = data[['Color_d', 'Cuteness_d', 'Emotions_d', 'Personal_Preference_d', 'Shape_d','Size_d']]
dependent_var = data['Chosen_Monster']

#adding intercept
intercept = sm.add_constant(independent_vars)

logistic_model = sm.Logit(dependent_var, independent_vars)
result = logistic_model.fit()

result.summary()
probabilities = result.predict(independent_vars)




Optimization terminated successfully.
         Current function value: 0.163041
         Iterations 9


0     0.000811
1     0.140401
2     0.999162
3     0.999251
4     0.991651
        ...   
86    0.759842
87    0.999765
88    0.914087
89    0.001106
90    0.060161
Length: 91, dtype: float64